# 第12章 pandas高级应用

## 12.1 分类数据

### 背景和目的

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [4]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [7]:
values = pd.Series([0, 1, 0, 0] * 2)

In [8]:
dim = pd.Series(['apple', 'orange'])

In [9]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [10]:
dim

0     apple
1    orange
dtype: object

In [11]:
dim.take(values)
#take方法存储Series数据
#用整数表示实际的分类，用编码（代码）而不是原始的分类，有主意提高性能
#使用字典的方法，构造键值对，键用整数，值代表实际的类别，将实际类别替换为相应的整数
#类似于sql中的外键，构造代码表并使用外键，可以减少重复的分类数据所占用的存储

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### pandas的分类类型

In [12]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [13]:
N = len(fruits)

In [14]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),#randoint方法从指定的范围内选出指定数量的整数
                   'weight': np.random.uniform(0, 4, size=N)},#类似上边
                  columns=['basket_id', 'fruit', 'count', 'weight'])#指定列及列名

In [15]:
df

,basket_id,fruit,count,weight
0,0,apple,4,0.504435
1,1,orange,14,1.694327
2,2,apple,11,1.972680
3,3,apple,9,3.882569
4,4,apple,14,0.948854
5,5,orange,12,0.185036
6,6,apple,3,0.047959
7,7,apple,13,2.902867


In [16]:
fruit_cat=df['fruit'].astype('category')
#astype参数指定为category后，fruit_cat值的类型就不是np数组，而是pandas.Categorical实例
#就可以调用它对df进行分类了

In [17]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [18]:
c = fruit_cat.values

In [19]:
type(c)

pandas.core.arrays.categorical.Categorical

In [20]:
type(fruit_cat)
#df的一列就是Series，类似于矩阵的一列就是向量

pandas.core.series.Series

In [21]:
c.categories
#该属性就是所有不同的值

Index(['apple', 'orange'], dtype='object')

In [22]:
c.codes 
#该属性就是各行的值对应的整数代码

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [23]:
df['fruit'] = df['fruit'].astype('category')
#将df的列使用astype方法指定为分类

In [24]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [25]:
df

,basket_id,fruit,count,weight
0,0,apple,4,0.504435
1,1,orange,14,1.694327
2,2,apple,11,1.972680
3,3,apple,9,3.882569
4,4,apple,14,0.948854
5,5,orange,12,0.185036
6,6,apple,3,0.047959
7,7,apple,13,2.902867


In [26]:
type(df['fruit'])

pandas.core.series.Series

In [27]:
df['fruit'].dtype

CategoricalDtype(categories=['apple', 'orange'], ordered=False)

In [28]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
#手动从其他对象直接创建pandas.Categorical实例

In [29]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [32]:
categories = ['foo', 'bar', 'baz']
#可以先指定用来分类的唯一值

In [33]:
codes = [0, 1, 2, 0, 0, 1]
#然后指定或从别处获取分类编码序列

In [34]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
#然后利用上述两个对象创建分类

In [35]:
my_cats_2
#分类变换不认定指定的分类顺序，而是取决于输入数据的顺序

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [36]:
ordered_cat = pd.Categorical.from_codes(codes, categories,ordered=True)
#orderd=True参数，设置分类的顺序

In [37]:
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [38]:
pd.Categorical.from_codes??
#(codes, categories=None, ordered=None, dtype=None)

In [39]:
my_cats_2.as_ordered()
#无序的分类可以使用as_ordered方法排序

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [40]:
my_cats_2.as_ordered??

### 用分类进行计算

In [41]:
np.random.seed(12345)

In [42]:
draws=np.random.randn(1000)

In [43]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [45]:
#draws.head() #AttributeError: 'numpy.ndarray' object has no attribute 'head'

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [58]:
bins=pd.qcut(draws,4)#使用qcut划分包含元素数量相等的四个区间，并用区间替代原始元素

In [59]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [60]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [61]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])#为区间改名，这样看起来更清楚，也利于生成汇总

In [62]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [63]:
pd.qcut??
#pd.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')

In [64]:
bins.codes[:10]


array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [65]:
bins = pd.Series(bins, name='quartile')

In [87]:
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index())

In [88]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [89]:
results = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index()
#没必要加括号的嘛
#先把array类型的draws转换为Series类型的对象
#然后按照已经转化为Series类型的bins进行分组
#分组后agg 'count', 'min', 'max'三个聚合函数
#最后使用reset_index重置索引

In [90]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [91]:
results1 = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max'])
#作为对比，不reset_index的结果

In [92]:
results1

,count,min,max
quartile,,,
Q1,250,-2.949343,-0.685484
Q2,250,-0.683066,-0.010115
Q3,250,-0.010032,0.628894
Q4,250,0.634238,3.927528


In [93]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [94]:
results1['count']
#results1['quartile'] 会报错，因为result1 中，quartile是作为行的索引出现的，而不是作为列出现

quartile
Q1    250
Q2    250
Q3    250
Q4    250
Name: count, dtype: int64

### 用分类提高性能

In [101]:
N=10000000

In [102]:
draws=pd.Series(['foo','bar','baz','qux']*(N//4))

In [103]:
draws

0          foo
1          bar
2          baz
3          qux
4          foo
5          bar
6          baz
7          qux
8          foo
9          bar
10         baz
11         qux
12         foo
13         bar
14         baz
15         qux
16         foo
17         bar
18         baz
19         qux
20         foo
21         bar
22         baz
23         qux
24         foo
25         bar
26         baz
27         qux
28         foo
29         bar
          ... 
9999970    baz
9999971    qux
9999972    foo
9999973    bar
9999974    baz
9999975    qux
9999976    foo
9999977    bar
9999978    baz
9999979    qux
9999980    foo
9999981    bar
9999982    baz
9999983    qux
9999984    foo
9999985    bar
9999986    baz
9999987    qux
9999988    foo
9999989    bar
9999990    baz
9999991    qux
9999992    foo
9999993    bar
9999994    baz
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: object

In [104]:
N//4

2500000

In [105]:
N

10000000

In [106]:
labels=pd.Series(['foo','bar','baz','qux']*(N//4))

In [107]:
categories=labels.astype('category')

In [108]:
labels.memory_usage()
#标签的内存使用量

80000080

In [109]:
categories.memory_usage()
#分类的内存使用量

10000272

In [110]:
%time _=labels.astype('category')
#转换分类的时间代价——这是一个一次性的代价，在处理大规模数据的时候，标签转为分类可以大量节省内存

Wall time: 402 ms


In [111]:
%time??
#为什么没有返回CUP时间之类的？

### 分类方法 

In [112]:
s= pd.Series(['a','b','c','d']*2)

In [114]:
cat_s=s.astype('category')

In [115]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [116]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [117]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [118]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [119]:
cat_s2 = cat_s.cat.set_categories(actual_categories)
#使用set_categories方法修改categories

In [120]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [121]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [122]:
cat_s2.value_counts()
#分类集改变了，即使数据没变，相关的统计信息也会变的

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [124]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [125]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [126]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

Category可以看作是一种新的数据类型，类似于统计学里的分类变量。类似于Python中的其他相似的数据类型，Category具有一些方法，例如，add_categories在已存在的分类后添加新的分类；as_ordered方法使分类有序，as_unordered使分类无序；remove_categories移除分类，设置任何被移除的值为null，remove_unused_categories移除任意不出现在数据中的分类值；rename_categories用指定的新分类的名字替换分类，但不改变分类的数目；reorder_categories可以改变结果，使分类有序；set_categories用指定的新分类的名字替换分类，可以添加或删除分类。

### 为建模创建虚拟变量

In [127]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [130]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [128]:
pd.get_dummies(cat_s)#pd.get_dummies方法将一维分类数据转换为DF数据

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 Groupby高级应用

### 分组转换和“解封”Groupby

In [132]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,'value': np.arange(12.)})

In [133]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [134]:
g = df.groupby('key').value

In [135]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [136]:
g.transform(lambda x: x.mean())
#transform方法，类似于apply，也可以传递函数
#它可以广播
#效果类似于sql的开窗函数

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [137]:
g.transform('mean')
#对于内置的聚合函数，可以直接传递函数名

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [140]:
g.transform('max')#每次只能传递一个参数

TypeError: f() takes 1 positional argument but 2 were given

In [143]:
g.transform(lambda x:x*2)#与apply函数类似，transform函数也会返回Series，长度与输入相等。

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [144]:
g.transform(lambda x: x.rank(ascending=False))#计算降序排名

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [146]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [147]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [148]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [151]:
%time g.transform('mean')

Wall time: 3 ms


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [155]:
%time g.mean()#内置的聚合函数通常比apply和transform快

Wall time: 2 ms


key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [152]:
%time normalized = (df['value'] - g.transform('mean')) / g.transform('std')

Wall time: 6 ms


In [153]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [156]:
%time g.transform(normalize)#??为什么会更快？？？

Wall time: 5 ms


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### 分组的时间重采样

In [157]:
N = 15

In [158]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [159]:
df = pd.DataFrame({'time': times, 'value': np.arange(N)})

In [160]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [163]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [164]:
df2 = pd.DataFrame({'time': times.repeat(3),'key': np.tile(['a', 'b', 'c'], N),'value': np.arange(N * 3.)})

In [165]:
df2

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0
9,2017-05-20 00:03:00,a,9.0


In [166]:
time_key = pd.TimeGrouper('5min')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


In [167]:
resampled = (df2.set_index('time').groupby(['key', time_key]) .sum())

In [168]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

## 12.3 链式编程技术

In [173]:
df.assign??

### 管道方法